In [10]:
from load_data import data, mice, phases

# List of animals in experiment
print(list(mice))

# experiment phases
print(phases.sections())
# start, end of each phase (as Unix time - https://en.wikipedia.org/wiki/Unix_time)
for phase in phases.sections():
    pass
    #print(phases.gettime(phase))
    
# Visits of a mouse to the rooms during one phase can be accesed like that:
phase = 'PHASE 1 dark'
mouse = list(mice)[0]
data.unmask_data()
data.mask_data(*phases.gettime(phase))
# Because of masking only visits starting in the given phase are returned.
start_times = data.getstarttimes(mouse)
end_times = data.getendtimes(mouse)
room_numbers = data.getaddresses(mouse)

for st, en, room in zip(start_times, end_times, room_numbers):
    pass
    #print("visit to room %d, starting %f, ending %f" %(room, st, en))



['0065-0136670531', '0065-0136651817', '0065-0136659459', '0065-0136653169', '0065-0136667521', '0065-0136655780', '0065-0136661759', '0065-0136665886', '0065-0136660676', '0065-0136673193', '0065-0136671473', '0065-0136659288']
['PHASE 1 dark', 'PHASE 1 light', 'PHASE 2 dark', 'PHASE 2 light', 'PHASE 3 dark', 'PHASE 3 light']


In [5]:
import csv
from load_data import data, mice, phases

room_time = {mouse: {room: 0 for room in range(1, 5)} for mouse in mice}
pair_time = {}

for phase in phases.sections():
    print(f"Processing {phase}...")
    data.unmask_data()
    data.mask_data(*phases.gettime(phase))
    
    mouse_visits = {}
    
    for mouse in mice:
        start_times = data.getstarttimes(mouse)
        end_times = data.getendtimes(mouse)
        room_numbers = data.getaddresses(mouse)
        
        visits = []
        for st, en, room in zip(start_times, end_times, room_numbers):
            visit_duration = en - st
            room_time[mouse][room] += visit_duration
            visits.append((st, en, room))
        
        mouse_visits[mouse] = visits
    
    for i, mouse1 in enumerate(mice):
        for mouse2 in list(mice)[i+1:]:
            pair_key = (mouse1, mouse2)
            pair_time[pair_key] = 0
            
            for st1, en1, room1 in mouse_visits[mouse1]:
                for st2, en2, room2 in mouse_visits[mouse2]:
                    if room1 == room2:
                        overlap_start = max(st1, st2)
                        overlap_end = min(en1, en2)
                        if overlap_start < overlap_end:
                            pair_time[pair_key] += overlap_end - overlap_start

def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours}h {minutes}m {seconds}s"
    
with open('room_time.csv', 'w', newline='') as csvfile:
    fieldnames = ['Mouse', 'Room', 'Time Spent']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for mouse, rooms in room_time.items():
        for room, time in rooms.items():
            formatted_time = format_time(time)
            writer.writerow({'Mouse': mouse, 'Room': room, 'Time Spent': formatted_time})

with open('pair_time.csv', 'w', newline='') as csvfile:
    fieldnames = ['Mouse 1', 'Mouse 2', 'Time Together']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for (mouse1, mouse2), time in pair_time.items():
        formatted_time = format_time(time)
        writer.writerow({'Mouse 1': mouse1, 'Mouse 2': mouse2, 'Time Together': formatted_time})

print("Results saved to room_time.csv and pair_time.csv.")


Processing PHASE 1 dark...
Processing PHASE 1 light...
Processing PHASE 2 dark...
Processing PHASE 2 light...
Processing PHASE 3 dark...
Processing PHASE 3 light...
Results saved to room_time.csv and pair_time.csv.
